# The Vector Space Model

We have seen how a single document could be converted into a _bag of words_ (or a bag of $n$-grams) representation. Now we will go one step further, converting an entire corpus of documents into tabular data.

## Term Frequencies

The bag of words representation gives us a mapping between words and their counts, such as `{..., "am": 3, "i": 71, "sam": 6, ...}`. To turn the bag of words into a vector of numbers, we can simply take the word counts, as follows:

| ... | i  | am | sam | ... |
|-----|----|----|-----|-----|
| ... | 71 |  3 |  6  | ... |

If we do this for each document in the corpus, and stack the rows, we obtain a table of numbers called the **term-frequency (TF) matrix**.

|        | ... | i  | am | sam | ... |
|--------|-----|----|----|-----|-----|
|**green_eggs_and_ham**| ... | 71 |  3 |  6  | ... |
|**cat_in_the_hat**| ... | 59 | 0 | 0 | ... |
|**fox_in_socks**| ... | 13 | 0 | 0 | ... |
|...|...|...|...|...|...|
|**one_fish_two_fish**| ... | 51 | 3 | 0 | ... |

The columns are all words (or _terms_) that appear in the corpus, which collectively make up the _vocabulary_. The idea of representing documents by a vector of numbers is called the _vector space model_.

Terminology note: we are use **term frequency (TF)** to refer to the *number* of times the term appears in the document. "Term frequency" is also sometimes computed as the *proportion* of words in the document that are equal to the terms (that is, the relative frequency of the term in the document). There are still other ways to define TF (or TF-IDF that we'll see below).

### Implementation from Scratch

Let's obtain the term-frequency matrix for the Dr. Seuss books. First, we read in the data.

In [1]:
import pandas as pd
import requests

seuss_dir = "http://dlsun.github.io/pods/data/drseuss/"
seuss_files = [
    "green_eggs_and_ham.txt", "cat_in_the_hat.txt", "fox_in_socks.txt",
    "hop_on_pop.txt", "horton_hears_a_who.txt", "how_the_grinch_stole_christmas.txt",
    "oh_the_places_youll_go.txt", "one_fish_two_fish.txt"
]

docs_seuss = pd.Series()
for file in seuss_files:
    response = requests.get(seuss_dir + file, "r")
    docs_seuss[file[:-4]] = response.text

docs_seuss

green_eggs_and_ham                I am Sam\n\nI am Sam\nSam I am\n\nThat Sam-I-a...
cat_in_the_hat                    The sun did not shine.\nIt was too wet to play...
fox_in_socks                      Fox\nSocks\nBox\nKnox\n\nKnox in box.\nFox in ...
hop_on_pop                        UP PUP Pup is up.\nCUP PUP Pup in cup.\nPUP CU...
horton_hears_a_who                On the fifteenth of May, in the jungle of Nool...
how_the_grinch_stole_christmas    Every Who\nDown in Whoville\nLiked Christmas a...
oh_the_places_youll_go            Congratulations!\nToday is your day.\nYou're o...
one_fish_two_fish                 One fish, two fish, red fish, blue fish,\nBlac...
dtype: object

In [30]:
import numpy as np

Now we apply the bag of words representation to the normalized text.

In [2]:
from collections import Counter

bag_of_words = (
    docs_seuss.
    str.lower().                  # convert all letters to lowercase
    str.replace("[^\w\s]", " ").  # replace non-alphanumeric characters by whitespace
    str.split()                   # split on whitespace
).apply(Counter)

bag_of_words

green_eggs_and_ham                {'i': 71, 'am': 3, 'sam': 3, 'that': 3, 'sam-i...
cat_in_the_hat                    {'the': 97, 'sun': 2, 'did': 8, 'not': 37, 'sh...
fox_in_socks                      {'fox': 11, 'socks': 8, 'box': 3, 'knox': 8, '...
hop_on_pop                        {'up': 4, 'pup': 7, 'is': 12, 'up.': 2, 'cup':...
horton_hears_a_who                {'on': 17, 'the': 96, 'fifteenth': 1, 'of': 37...
how_the_grinch_stole_christmas    {'every': 5, 'who': 10, 'down': 9, 'in': 15, '...
oh_the_places_youll_go            {'congratulations!': 1, 'today': 2, 'is': 7, '...
one_fish_two_fish                 {'one': 10, 'fish,': 7, 'two': 2, 'red': 1, 'b...
dtype: object

To turn this into a term-frequency matrix, we need to make a `DataFrame` out of it, where each column represents a word and each row a document---and each entry is the count of that word in the document.

In [3]:
tf = pd.DataFrame(list(bag_of_words))
tf

,i,am,sam,that,sam-i-am,sam-i-am!,do,not,like,you,...,game?,gack,park,"home,",clark.,grow,sleep,zeep.,gone.,one.
0,71,3.0,3.0,3,4.0,2.0,35.0,65,44.0,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,48,NaN,NaN,20,NaN,NaN,15.0,37,13.0,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,NaN,NaN,1,NaN,NaN,7.0,1,1.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,1.0,NaN,4,NaN,NaN,NaN,2,6.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18,1.0,NaN,31,NaN,NaN,1.0,6,NaN,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,NaN,NaN,13,NaN,NaN,2.0,1,2.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,NaN,NaN,11,NaN,NaN,4.0,6,1.0,43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,48,3.0,NaN,1,NaN,NaN,11.0,10,21.0,24,...,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0


This matrix is full of missing numbers. A missing number means that the word did not appear in that document. In other words, a count of `NaN` really means a count of 0. So it makes sense in this situation to replace the `NaN`s by 0s.

In [4]:
tf = tf.fillna(0)
tf

,i,am,sam,that,sam-i-am,sam-i-am!,do,not,like,you,...,game?,gack,park,"home,",clark.,grow,sleep,zeep.,gone.,one.
0,71,3.0,3.0,3,4.0,2.0,35.0,65,44.0,21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,48,0.0,0.0,20,0.0,0.0,15.0,37,13.0,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9,0.0,0.0,1,0.0,0.0,7.0,1,1.0,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,1.0,0.0,4,0.0,0.0,0.0,2,6.0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,18,1.0,0.0,31,0.0,0.0,1.0,6,0.0,27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,0.0,0.0,13,0.0,0.0,2.0,1,2.0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2,0.0,0.0,11,0.0,0.0,4.0,6,1.0,43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,48,3.0,0.0,1,0.0,0.0,11.0,10,21.0,24,...,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0


### Implementation using `scikit-learn`

We could have also used the `CountVectorizer` in `scikit-learn` to obtain the term-frequency matrix. This vectorizer is fit to a list of the documents in the corpus. By default, it converts all letters to lowercase and strips punctuation, although this behavior can be customized using the `strip_accents=` and `lowercase=` parameters.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
vec.fit(docs_seuss) # This determines the vocabulary.
tf_sparse = vec.transform(docs_seuss)

tf_sparse

<8x1344 sparse matrix of type '<class 'numpy.int64'>'
	with 2308 stored elements in Compressed Sparse Row format>

Notice that `CountVectorizer` returns the term-frequency matrix, not as a `DataFrame` or even as a `numpy` array, but as a `scipy` sparse matrix. A _sparse matrix_ is one whose entries are mostly zeroes. For example,

$$ \begin{pmatrix} 0 & 0 & 0 & 0 & 0 \\ 1.7 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & -0.8 & 0 \end{pmatrix} $$

is an example of a sparse matrix. Instead of storing 20 values (most of which are equal to 0), we can simply store the locations of the non-zero entries and their values:

- $(1, 0) \rightarrow 1.7$
- $(3, 3) \rightarrow -0.8$

All other entries of the matrix are assumed to be zero. This representation offers substantial memory savings when there are only a few non-zero entries. (But if not, then this representation can actually be more expensive.) Term-frequency matrices are usually sparse because most words do not appear in all documents.

The `scipy` sparse matrix format is used to store sparse matrices. If necessary, a `scipy` sparse matrix can be converted to a `numpy` matrix using the `.todense()` method.

In [6]:
tf_sparse.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 3, 1, 1]])

We can further convert this `numpy` matrix to a `pandas` `DataFrame`. To make the column names descriptive, we call the `.get_feature_names_out()` method of the `CountVectorizer`, which returns a list of the words in the order that they appear in the matrix.

In [7]:
pd.DataFrame(
    tf_sparse.todense(),
    columns=vec.get_feature_names_out()
)

,12,56,98,able,about,act,afraid,after,afternoon,again,...,yop,yopp,you,young,your,yourself,yourselves,zans,zeds,zeep
0,0,0,0,0,0,0,0,0,0,0,...,0,0,34,0,0,0,0,0,0,0
1,0,0,0,0,3,0,0,1,0,0,...,0,0,34,0,8,0,0,0,0,0
2,0,0,0,0,2,0,0,0,0,0,...,0,0,8,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,2,0,0,...,0,0,2,0,0,0,0,0,0,0
4,1,1,0,1,1,0,0,4,2,1,...,0,3,47,5,7,0,1,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,2,1,0,0,0,0,0,0
6,0,0,1,0,1,1,2,0,0,0,...,0,0,85,0,20,2,0,0,0,0
7,0,0,0,0,1,0,0,0,0,2,...,1,0,24,0,9,0,0,3,1,1


The term-frequency matrix that `CountVectorizer` produced is not exactly the same as the matrix that we produced ourselves using just `pandas`. Although the two matrices have the same number of rows (8, corresponding to the number of documents in the corpus), they have a different number of columns. It appears that `CountVectorizer` had a vocabulary that was 11 words smaller (1344 words instead of 1355). We can determine exactly which 11 words these are, by taking the set difference:

In [8]:
set(tf.columns) - set(vec.get_feature_names_out())

{'yipping!',
 'grinch.',
 'still.',
 'things."',
 "i've",
 'christmas!',
 'gump.',
 'you!"',
 'whos,',
 'oh!',
 'roof,',
 'win?',
 'broom.',
 'brown?',
 'dope!',
 'so...',
 'packages,',
 'you,"',
 'say?',
 'fun?',
 'sacks,',
 'joys...',
 '"and',
 'headed,',
 'know,',
 'kettles!',
 'nonsense!',
 'pop.',
 'small,',
 'shirking?"',
 'trick,',
 '"if',
 'sally.',
 'pink.',
 'all!"',
 'there.',
 'net,',
 'around."',
 'liar.',
 'bones,',
 'talking!',
 'marked.',
 'night.he',
 'so!',
 'nose.',
 'loudly.',
 'now.',
 'why:',
 'said...',
 'funny!"',
 'yell.',
 'down,',
 'claus,',
 'tinsel!',
 'nick!"',
 'near?',
 'i',
 'brush!',
 'succeed?',
 "ben's",
 'out?',
 'darked.',
 "mother's",
 'heights.',
 'bicycles!',
 '"boil',
 'hour!',
 'bit!"',
 'town,',
 "trees'",
 'then?',
 '"there\'s',
 'done.',
 'side."',
 "horton's",
 'you.',
 'be.',
 'how?"',
 'said.',
 'step.',
 'blocks.',
 'chewing!',
 'bed.',
 'safe,',
 'lighted.',
 'now!',
 'so...get',
 'town.',
 'trees.',
 'cow?',
 'me,',
 '"me,',
 'heads.'

We see that all of the words that `CountVectorizer` missed were one-character long. By default, `CountVectorizer` only retains words that are at least 2 characters long. This behavior can be customized using regular expressions in the `token_pattern=` parameter. (By default, scikit-Learn strips punctuation and
converts all characters to lowercase, but you can also customize that behavior using `token_pattern`.) However, since 1-letter words are usually not useful for analysis anyway, this is a minor technical point. We won't cover regular expressions in any detail.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
vec.fit(docs_seuss) # This determines the vocabulary.
tf_sparse = vec.transform(docs_seuss)

tf_sparse

<8x1355 sparse matrix of type '<class 'numpy.int64'>'
	with 2345 stored elements in Compressed Sparse Row format>

In [10]:
pd.DataFrame(
    tf_sparse.todense(),
    columns=vec.get_feature_names_out()
)["am"]

0    16
1     0
2     0
3     1
4     1
5     0
6     0
7     3
Name: am, dtype: int64

`CountVectorizer` can even count $n$-grams by specify `ngram_range`. If we wanted bigrams, then we would specify `ngram_range=(2, 2)`.

In [11]:
vec = CountVectorizer(ngram_range=(2, 2))
vec.fit(docs_seuss)
tf_sparse_bigram = vec.transform(docs_seuss)

tf_sparse_bigram

<8x5846 sparse matrix of type '<class 'numpy.int64'>'
	with 6459 stored elements in Compressed Sparse Row format>

In [12]:
pd.DataFrame(
    tf_sparse_bigram.todense(),
    columns=vec.get_feature_names_out()
)

,12 very,56 the,98 and,able to,about he,about it,about some,about to,about tweetle,about when,...,your towns,your way,your yapping,yourself any,yourself is,yourselves heard,zans for,zans zans,zeds they,zeep today
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,1,0,0,0,1,0,0,...,1,0,1,0,0,1,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,1,0,0,0,1,0,0,0,...,0,2,0,1,1,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,2,1,1,1


Note that there are about 5800 bigrams. A data frame with 8 rows and 5800 columns has around 50000 entries. However, only about 6500 of these entries are non-zero. This is why sparse matrices are vital in text processing.

In [13]:
# number of non-zero values in the sparse matrix.
tf_sparse_bigram.count_nonzero()

6459

If we wanted unigrams (i.e., individual words), bigrams, and trigrams then we would specify `ngram_range=(1, 3)`.

In [14]:
vec = CountVectorizer(ngram_range=(1, 3))
vec.fit(docs_seuss)
vec.transform(docs_seuss)

<8x14918 sparse matrix of type '<class 'numpy.int64'>'
	with 16560 stored elements in Compressed Sparse Row format>

## TF-IDF

The problem with term frequencies (TF) is that common words like "the" and "that" tend to have high counts and dominate. A better indicator of whether two documents are similar is if they share rare words. For example, the word "eat" only appears in two of the Dr. Seuss stories. The presence of that word in two documents is a strong indicator that the documents are similar, so we should give more weight to terms like it.

This is the idea behind TF-IDF. We take each term frequency and re-weight it according to how many documents that term appears in, i.e., the **document frequency**. Since we want words that appear in fewer documents to get more weight, we take the **inverse document frequency** (IDF).  We take the logarithm of IDF because the distribution of IDFs is heavily skewed to the right. (Remember the discussion about transforming data from earlier.) When dealing with IDF, the base-10 logarithm ($\log_{10}$) is typically used. So in the end, the formula for IDF for term $t$ in corpus $D$ is:

$$ \textrm{idf}(t, D) = \log_{10} \left(\frac{\textrm{\# of documents in corpus}}{\textrm{\# of documents in corpus that contain term } t}\right) = \log_{10} \left(\frac{|D|}{|d \in D: t \in d|}\right). $$

(Sometimes, $1$ will be added to the denominator to prevent division by zero, if there are terms in the vocabulary that do not appear in the corpus.)

To calculate **TF-IDF** of term $t$ in document $d$ of corpus $D$, we simply multiply the term frequencies by the inverse document frequencies:

$$ \textrm{tf-idf}(d, t, D) = \textrm{tf}(d, t) \cdot \textrm{idf}(t, D). $$

Notice that unlike TF, the TF-IDF representation of a given document depends on the entire corpus of documents.

### Implementation from Scratch

Let's first see how to calculate TF-IDF from scratch, using the term-frequency matrix we obtained above. (We will typically use scikit-learn, which we'll see how to do below. We just implement from scratch to make the process a little more concrete.)

First we get the document frequencies: how many documents does each word appear in? (Since our corpus has 8 documents, each DF will be between 1 and 8.) Note `(tf > 0)` below creates a data frame of Booleans, with one column for each word, and we can sum each column over all the documents in the corpus to get the DFs.

In [15]:
# Get document frequencies
# (How many documents does each word appear in?)
df = (tf > 0).sum(axis=0)
df

i           8
am          4
sam         1
that        8
sam-i-am    1
           ..
grow        1
sleep       1
zeep.       1
gone.       1
one.        1
Length: 2264, dtype: int64

Now we get the IDFs of each word in the corpus.

In [16]:
# Get IDFs
idf = np.log(len(tf) / df)
idf

i           0.000000
am          0.693147
sam         2.079442
that        0.000000
sam-i-am    2.079442
              ...   
grow        2.079442
sleep       2.079442
zeep.       2.079442
gone.       2.079442
one.        2.079442
Length: 2264, dtype: float64

Finally we multiply the TF and IDF to get the TF-IDF of each word in the corpus

In [17]:
# Calculate TF-IDFs
tf_idf = tf * idf
tf_idf

,i,am,sam,that,sam-i-am,sam-i-am!,do,not,like,you,...,game?,gack,park,"home,",clark.,grow,sleep,zeep.,gone.,one.
0,0.0,2.079442,6.238325,0.0,8.317766,4.158883,4.673599,0.0,5.875381,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.000000,0.000000,0.0,0.000000,0.000000,2.002971,0.0,1.735908,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.934720,0.0,0.133531,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.693147,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.801188,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.693147,0.000000,0.0,0.000000,0.000000,0.133531,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.267063,0.0,0.267063,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.534126,0.0,0.133531,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,2.079442,0.000000,0.0,0.000000,0.000000,1.468845,0.0,2.804159,0.0,...,2.079442,2.079442,2.079442,2.079442,2.079442,2.079442,4.158883,2.079442,2.079442,2.079442


### Implementation using `scikit-learn`

We will not generally implement TF-IDF from scratch, like we did above. Instead, we will use Scikit-Learn's `TfidfVectorizer`, which operates similarly to `CountVectorizer`, except that it returns a matrix of the TF-IDF weights.

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", smooth_idf = False, norm=None)
vec.fit(docs_seuss)
tf_idf_sparse = vec.transform(docs_seuss)
tf_idf_sparse

<8x1355 sparse matrix of type '<class 'numpy.float64'>'
	with 2345 stored elements in Compressed Sparse Row format>

In [19]:
pd.DataFrame(
    tf_idf_sparse.todense(),
    columns=vec.get_feature_names_out()
)

,12,3,4,56,6,98,a,able,about,act,...,yop,yopp,you,young,your,yourself,yourselves,zans,zeds,zeep
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,59.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,34.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34.0,0.000000,4.410011,0.000000,...,0.000000,0.000000,34.0,0.000000,11.760029,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.0,0.000000,2.940007,0.000000,...,0.000000,0.000000,8.0,0.000000,1.470004,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,2.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,3.079442,0.000000,0.000000,3.079442,3.079442,0.000000,48.0,3.079442,1.470004,0.000000,...,0.000000,9.238325,47.0,11.931472,10.290025,0.000000,3.079442,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,2.0,2.386294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,3.079442,3.079442,0.000000,0.000000,3.079442,24.0,0.000000,1.470004,3.079442,...,0.000000,0.000000,85.0,0.000000,29.400073,6.158883,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.0,0.000000,1.470004,0.000000,...,3.079442,0.000000,24.0,0.000000,13.230033,0.000000,0.000000,9.238325,3.079442,3.079442


You might notice that the skikit-learn results do not match what we computed from scratch. The reason is the scikit-learn IDF adds 1 to the definition of IDF we presented above.

In [20]:
# Calculate TF-IDFs
tf * (1 + idf)

,i,am,sam,that,sam-i-am,sam-i-am!,do,not,like,you,...,game?,gack,park,"home,",clark.,grow,sleep,zeep.,gone.,one.
0,71.0,5.079442,9.238325,3.0,12.317766,6.158883,39.673599,65.0,49.875381,21.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,48.0,0.000000,0.000000,20.0,0.000000,0.000000,17.002971,37.0,14.735908,30.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,9.0,0.000000,0.000000,1.0,0.000000,0.000000,7.934720,1.0,1.133531,7.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2.0,1.693147,0.000000,4.0,0.000000,0.000000,0.000000,2.0,6.801188,2.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,18.0,1.693147,0.000000,31.0,0.000000,0.000000,1.133531,6.0,0.000000,27.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,6.0,0.000000,0.000000,13.0,0.000000,0.000000,2.267063,1.0,2.267063,2.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,2.0,0.000000,0.000000,11.0,0.000000,0.000000,4.534126,6.0,1.133531,43.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,48.0,5.079442,0.000000,1.0,0.000000,0.000000,12.468845,10.0,23.804159,24.0,...,3.079442,3.079442,3.079442,3.079442,3.079442,3.079442,6.158883,3.079442,3.079442,3.079442


This differnece is a minor techical detail that we won't worry about. (There are still [other ways to define TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)). We'll just use the scikit-learn results. But the following code shows that once we add 1 to our definition of IDF, the "from scratch" calculation of TF-IDF matches scikit-learn (because the maximum absolute difference between the two is 0)

In [31]:
abs((pd.DataFrame(
    tf_idf_sparse.todense(),
    columns=vec.get_feature_names_out()
) - tf * (1 + idf))).max().max()

49.27106466687738

## Cosine Similarity

We now have a representation of each text document as a vector of numbers. Each number can either be a term frequency or a TF-IDF weight. We can visualize each vector as an arrow in a high-dimensional space, where each dimension represents a word. The magnitude of the vector along a dimension represents the "frequency" (TF or TF-IDF) of that word in the document. For example, if our vocabulary only contains two words, "i" and "sam", then the arrows shown below might represent two documents:

<img src="https://github.com/dlsun/pods/blob/master/10-Textual-Data/vector_space.png?raw=1" width="300"/>

How do we tell if two documents are similar? We need some way to measure the distance between two documents (i.e., two vectors). We could use Euclidean distance, as we have done in the past.

<img src="https://github.com/dlsun/pods/blob/master/10-Textual-Data/vector_space_euclidean.png?raw=1" width="300"/>

But Euclidean distance does not make sense for TF or TF-IDF vectors. To see why, consider the two documents:

1. "I am Sam."
2. "I am Sam. Sam I am."

The two documents are obviously very similar. But the vector for the second is twice as long as the vector for the first because the second document has twice as many occurrences of each word. This is true whether we use TF or TF-IDF weights. If we calculate the Euclidean distance between these two vectors, then they will seem quite far apart.

<img src="https://github.com/dlsun/pods/blob/master/10-Textual-Data/vector_space_example.png?raw=1" width="300"/>

With TF and TF-IDF vectors, the distinguishing property is their _direction_. Because the two vectors above point in the same direction, they are similar. We need a distance metric that measures how different their directions are. A natural way to measure the difference between the directions of two vectors is the angle between them.

<img src="https://github.com/dlsun/pods/blob/master/10-Textual-Data/vector_space_cosine.png?raw=1" width="300"/>

The cosine of the angle between two vectors ${\bf a}$ and ${\bf b}$ can be calculated as:

$$ \cos \theta = \frac{\sum a_j b_j}{\sqrt{\sum a_j^2} \sqrt{\sum b_j^2}}. $$

(The numerator is the _dot product_ of vectors ${\bf a}$ and ${\bf b}$; the denominator is the product of their lengths.) Although it is possible to work out the angle $\theta$ from this formula, it is more common to report $\cos\theta$ as a measure of similarity between two vectors. This similarity metric is called **cosine similarity**. Notice that when the angle $\theta$ is close to 0 (i.e., when the two vectors point in nearly the same direction), the value of $\cos\theta$ is high (close to 1.0, which is the maximum possible value).

The cosine _distance_ is defined as 1 minus the similarity. This makes it so that 0 means that the two vectors point in the same direction:

$$ d_{\cos}({\bf a}, {\bf b}) = 1 - \cos(\theta({\bf a}, {\bf b})) = 1 - \frac{\sum a_j b_j}{\sqrt{\sum a_j^2} \sqrt{\sum b_j^2}}. $$

### Implementation from Scratch

Let's calculate the cosine similarity between document 0 (_Green Eggs and Ham_) and document 2 (_Fox in Socks_) using the TF-IDF representation. (Again, we'll typically use scikit-learn, but we'll do it from scratch first to see that scikit-learn matches.)

In [22]:
# Calculate the numerator.
a = tf_idf_sparse[0, :]
b = tf_idf_sparse[2, :]
dot = a.multiply(b).sum()

# Calculate the terms in the denominator.
a_len = np.sqrt(a.multiply(a).sum())
b_len = np.sqrt(b.multiply(b).sum())

# Cosine similarity is their ratio.
dot / (a_len * b_len)

0.1258410993578232

These two vectors are not very similar, as evidenced by their low cosine similarity (close to 0.0). Let's try to find the most similar documents in the corpus to _Green Eggs and Ham_---in other words, its nearest neighbors. To do this, we will take advantage of _broadcasting_: we will multiply a TF-IDF vector (representing document 0) by the entire TF-IDF matrix and calculate the sum over the columns. This will give us a vector of dot products.

In [23]:
# Calculate the numerators.
a = tf_idf_sparse[0, :]
B = tf_idf_sparse
dot = a.multiply(B).sum(axis=1)
dot

matrix([[49399.86992009],
        [21095.1096498 ],
        [ 5878.95362908],
        [ 2528.95862343],
        [15276.15082587],
        [ 8541.36255536],
        [ 8917.18862756],
        [14519.02087512]])

In [24]:
# Calculate the denominators.
a_len = np.sqrt(a.multiply(a).sum())
b_len = np.sqrt(B.multiply(B).sum(axis=1))
print(a_len)
b_len

222.26081508014192


matrix([[222.26081508],
        [239.32261163],
        [210.19124777],
        [ 77.44263097],
        [276.89867775],
        [231.2954469 ],
        [162.08949899],
        [177.22694181]])

In [25]:
# Calculate their ratio to obtain cosine similarities.
dot / (a_len * b_len)

matrix([[1.        ],
        [0.39658397],
        [0.1258411 ],
        [0.14692602],
        [0.24821622],
        [0.16614879],
        [0.24751993],
        [0.36859096]])

Now let's put this matrix into a `DataFrame` so that we can easily sort the values in descending order.

In [26]:
cos_similarities = pd.DataFrame(dot / (a_len * b_len))[0]
most_similar = cos_similarities.sort_values(ascending=False)
most_similar

0    1.000000
1    0.396584
7    0.368591
4    0.248216
6    0.247520
5    0.166149
3    0.146926
2    0.125841
Name: 0, dtype: float64

Of course, the most similar document in the corpus to _Green Eggs and Ham_ (with a perfect cosine similarity of 1.0) is itself. But the next most similar text is _The Cat in the Hat_ (the document with index 1).

### Implementation using scikit-learn

It is also possible to calculate cosine similarities/distances in `scikit-learn` using the same API that we used to calculate Euclidean (and Manhattan) distances earier.

In [27]:
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

cosine_similarity(tf_idf_sparse)

array([[1.        , 0.39658397, 0.1258411 , 0.14692602, 0.24821622,
        0.16614879, 0.24751993, 0.36859096],
       [0.39658397, 1.        , 0.17904195, 0.31095348, 0.55961395,
        0.49882428, 0.41849834, 0.62425905],
       [0.1258411 , 0.17904195, 1.        , 0.08946843, 0.18424177,
        0.1293746 , 0.15848004, 0.19644695],
       [0.14692602, 0.31095348, 0.08946843, 1.        , 0.23619993,
        0.20138896, 0.14432695, 0.34237812],
       [0.24821622, 0.55961395, 0.18424177, 0.23619993, 1.        ,
        0.54243474, 0.45903587, 0.48027592],
       [0.16614879, 0.49882428, 0.1293746 , 0.20138896, 0.54243474,
        1.        , 0.29141396, 0.37161018],
       [0.24751993, 0.41849834, 0.15848004, 0.14432695, 0.45903587,
        0.29141396, 1.        , 0.39016939],
       [0.36859096, 0.62425905, 0.19644695, 0.34237812, 0.48027592,
        0.37161018, 0.39016939, 1.        ]])

The $(i, j)$th entry of this matrix represents the cosine similarity between the $i$th and $j$th documents. So the first row of this matrix contains the similarities between _Green Eggs and Ham_ and the other documents in the corpus. Check that these numbers match the ones we obtained manually.

In [28]:
cosine_similarity(tf_idf_sparse)[0]

array([1.        , 0.39658397, 0.1258411 , 0.14692602, 0.24821622,
       0.16614879, 0.24751993, 0.36859096])